<a href="https://colab.research.google.com/github/etterguillaume/PIMPN/blob/master/PIMPIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PIMPN (Python Integrated Miniscope Pipeline Notebook)**

**Version: 0.8.1**

This Colab Notebook has been designed to analyze calcium imaging and behavior videos directly on the cloud. It relies on a Dropbox API, CaImAn, which itself uses NormCorre for motion correction as well as Constrained Non-Negative Matrix Factorization (CNMF/CNMFE) for source extraction. Behavior video can be analyzed automatically using DeepLabCut and your own DeepPose model (currently working on making pre-trained models available).

# Features
- **Fully automatic and unsupervised:** once you found parameters that work well for your conditions, and trained deep learning models that suit your data (pre-trained models also available), you can run your analyzes in one click
- **Tailored for miniscope data**: Registers miniscope timestamps and informations about your experiment (subject name, date of experiment), dedicated to multiple .avi files although works with other formats (hdf5, tiff).
- **Cloud based**: Google Colab will temporarily allocate a virtual machine to perform the analysis job. No need to download data on your local computer, and save computing for other tasks! Analyzed data is transfered back to your cloud server.
- **Compatibility:** the data generated by the analysis can be saved in a matlab file that is compatible with [MiniscopeAnalysis](https://github.com/etterguillaume/MiniscopeAnalysis), but also hdf5 files that can be organized the way you like.

**DISCLAIMER**
PIMPIN relies on a powerful Dropbox API that allows to interact with your cloud repositories (dowload, upload, list or remove files). While it is design to facilitate access, downloads and uploads, beware of misuses. If you plan on sharing this notebook, do not forget to remove your Dropbox access token unless you wish to grant access to your data. You can also enter your gmail credentials to get notified when your analysis is done. Here again, don't forget to remove this info when sharing the notebook.

Copyright © Guillaume Etter 2019

This program is free software; you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation; either version 3 of the License, or any later version.

Contact: etterguillaume@gmail.com

# Get informations about the virtual machine being used

In [0]:
# Disk information
!df -h
# CPU information
!lscpu | grep "MHz"
# If using a GPU
!nvidia-smi -L
!nvcc --version
# Memory information
!cat /proc/meminfo | grep 'MemAvailable'

# Parameters

In [0]:
access_token = '' # Get your access token from https://www.dropbox.com/developers/apps
path_to_analyze = ''
analyze_behavior = True
path_to_DLC_model = '' # Change yours here. Should be a zip file containing your full, trained model
spatial_downsampling = 3 # Drastically speeds up processing. 2-3 recommended
isnonrigid = False
path_to_results = '' # Where to save the data
alert_gmail = '' # You can leave your Gmail adress to be notified when your analysis is done
alert_gmail_password = '' # Password to your Gmail account

print('Parameters saved. Ready to start analyzing')

# Install and import dependencies

In [0]:
from datetime import datetime
import scipy.io as sio
import re
import os
import h5py
import csv
import tensorflow as tf
import time
import logging
import zipfile
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('default')
import numpy as np
from moviepy.editor import *
import smtplib

In [0]:
# Install Dropbox
!pip install dropbox
import dropbox

In [0]:
# Install CaImAn
#!git clone https://github.com/flatironinstitute/CaImAn.git
!git clone https://github.com/etterguillaume/CaImAn.git
%cd '/content/CaImAn/'
!pip install -e .

!pip install tifffile
!pip install ipyparallel
!pip install peakutils

%cd '/content/CaImAn/'
!python caimanmanager.py install --inplace

!export MKL_NUM_THREADS=1
!export OPENBLAS_NUM_THREADS=1

# This is for GPU accelaration
!pip install pycuda
!pip install scikit-cuda

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
import peakutils

# Access DropBox
This creates a dbx Dropbox object. It necessitates that you enter a valid token earlier while setting up parameters

In [0]:
dbx = dropbox.Dropbox(access_token)
print('Connected to Dropbox')

# Get information from the experiment
This is particularily important to register the date/time and name of the experiment, which are also use to create specific analysis folders containing all the analyzed data (calcium imaging and behavior). Additionnaly, timestamps are retrieved and allow to realign calcium imaging with behavior videos

In [0]:
now = datetime.now()
analysis_time = now.strftime("%Y-%m-%d %H:%M") # This is to register when the analysis was performed
print('Analysis started on ' + analysis_time)

analysis_start = time.time() # This is to register the time spent analyzing

# Open dat file here 
try:
  metadata, res = dbx.files_download(path_to_analyze + '/' + 'settings_and_notes.dat')
  f = open('settings_and_notes.dat','wb')
  f.write(res.content)
  f.close
except FileExistsError:
  print('Error: settings_and_notes.dat could not be found')

with open('settings_and_notes.dat') as f:
  for i, line in enumerate(f):
    if i == 0:
      line = str(f.readline())
      line = line.split('\t')
      experimentName = line[0]
      
print('Name of the experiment to analyze: ' + experimentName)

dirExperimentName = '/content/' + experimentName
 
try:
    # Create target Directory
    os.mkdir(dirExperimentName)
    print("Directory " , dirExperimentName ,  " Created") 
except FileExistsError:
    print("Directory " , dirExperimentName ,  " already exists")


# Extract the date/time of the experiment and save as a timestamp variable
The date/time values are extracted from the automatic DAQ folder organization (might not work if you renamed your folders)

In [0]:
try:
  from datetime import datetime
  splitname = str.split(path_to_analyze, '/')

  dateStrPart = splitname[-2]
  timeStrPart = splitname[-1]

  date_result = str.split(dateStrPart, '_')
  month = int(date_result[0])
  day = int(date_result[1])
  year = int(date_result[2])

  timeStrPart = re.sub('[HSM]','', timeStrPart)
  time_result = str.split(timeStrPart,'_')
  
  hour = int(time_result[0])
  minute = int(time_result[1])
  seconds = int(time_result[2])
  
  experiment_timestamp = datetime.timestamp(datetime(year,month,day,hour,minute,seconds))
  #dateNum = date.toordinal(date(year,month,day,hour,minute,seconds))

except:
  print('Could not retrieve date information')

# Access Dropbox and download miniscope video files


In [0]:
sessionFilesResponse = dbx.files_list_folder(path_to_analyze)
filesList = []
msFileList = []

for file in sessionFilesResponse.entries:
  filesList.append(file.name)

for i in filesList[:]:
  if i.startswith('ms') and i.endswith('.avi'):
    msFileList.append(i)
    
msFileList = sorted(msFileList, key=lambda x: int(re.sub('[msCam.avi]','', x)))

# FOR Quick BETA TESTING
#msFileList = msFileList[0:2]

print('Miniscope files in folder:')
print(msFileList)

if len(msFileList) == 0:
  print("No miniscope avi files found")

In [0]:
print('Downloading miniscope videos from Dropbox.....')
for file in msFileList:
    f = open(dirExperimentName + '/' + file,"wb+") 
    metadata, res = dbx.files_download(path_to_analyze + '/' + file)
    f.write(res.content)
    f.close()
    print(path_to_analyze + '/' + file + '.....done!')
print('Done downloading miniscope videos!')

In [0]:
# Create a list of files downloaded on the Colab virtual machine
msLocalFileList = [dirExperimentName + '/' + s for s in msFileList]
print(msLocalFileList)

In [0]:
# Visualize a couple of frames here for sanity check
clip = VideoFileClip(msLocalFileList[0])
clip.save_frame(dirExperimentName + '/' + 'frame.png')

img=mpl.image.imread(dirExperimentName + '/' + 'frame.png')
imgplot = plt.imshow(img); plt.title('Original size')

# Downsample the videos

In [0]:
for video in msLocalFileList:
  clip = VideoFileClip(video)
  resized_clip = clip.resize(1/spatial_downsampling)
  os.remove(video)
  resized_clip.write_videofile(video,codec='rawvideo')

In [0]:
# You can use this cell to inspect the downsample video
preview_video = False
if preview_video:
  clip = VideoFileClip(msVideoFilePath) # Play the first video
  ipython_display(clip)

In [0]:
# Make sure the video has been resized
clip = VideoFileClip(msLocalFileList[0])
clip.save_frame(dirExperimentName + '/' + 'downsampled_frame.png')

img=mpl.image.imread(dirExperimentName + '/' + 'downsampled_frame.png')
imgplot = plt.imshow(img); plt.title('Downsampled size')

In [0]:
fnames = msLocalFileList

In [0]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

# Set parameters for motion correction
Ideally, optimize these for your datasets then stick to these values

In [0]:
# dataset dependent parameters
frate = 30                       # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = False         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (3, 3)       # size of high pass spatial filtering, used in 1p data
max_shifts = (5, 5)      # maximum allowed rigid shift
strides = (48, 48)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (24, 24)      # overlap between patches (size of patch strides+overlaps)
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries
use_cuda = True         # Set to True in order to use GPU
only_init_patch = True
memory_fact = 0.8

mc_dict = {
    #'fnames': fnames,
    'fr': frate,
    'niter_rig': 1,
    'splits_rig': 20,  # for parallelization split the movies in  num_splits chuncks across time
    # if none all the splits are processed and the movie is saved
    'num_splits_to_process_rig': None, # intervals at which patches are laid out for motion correction            
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan,
    'use_cuda' : use_cuda,
    'only_init_patch' : only_init_patch,
    'memory_fact': memory_fact
}

opts = params.CNMFParams(params_dict=mc_dict)

# Perform motion correction (might take a while)

In [0]:
start = time.time() # This is to keep track of how long the analysis is running
if motion_correct:
    # do motion correction rigid
    mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))
    mc.motion_correct(save_movie=True)
    fname_mc = mc.fname_tot_els if pw_rigid else mc.fname_tot_rig
    
end = time.time()

print(end - start)
print('Motion correction has been done!')

# Plot the motion corrected template and associated shifts

In [0]:
%matplotlib inline
if motion_correct and not pw_rigid:
  plt.figure(figsize=(10,20))
  plt.subplot(2, 1, 1); plt.imshow(mc.total_template_rig);  # % plot template
  plt.subplot(2, 1, 2); plt.plot(mc.shifts_rig)  # % plot rigid shifts
  plt.legend(['x shifts', 'y shifts'])
  plt.xlabel('frames')
  plt.ylabel('pixels')

# Map the motion corrected video to memory

In [0]:
if motion_correct:  
    if pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                     np.max(np.abs(mc.y_shifts_els)))).astype(np.int)
    else:
        bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(np.int)

    bord_px = 0 if border_nan is 'copy' else bord_px
    fname_new = cm.save_memmap(fname_mc, base_name='memmap_', order='C',
                               border_to_0=bord_px)
    
else:  # if no motion correction just memory map the file
    fname_new = cm.save_memmap(fnames, base_name='memmap_',
                               order='C', border_to_0=0, dview=dview)
    
print('Motion corrected video has been mapped to memory')

In [0]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='F')

In [0]:
#%% restart cluster to clean up memory
cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

# Perform a projection of correlated pixels (and associated signal-to-noise ratio) in motion corrected video
This is important to assess the amounts of local correlations and peak-to-noise ratio as well as seed/initialize CNMFe

In [0]:
# Compute some summary images (correlation and peak to noise) while downsampling temporally 5x to speedup the process and avoid memory overflow
cn_filter, pnr = cm.summary_images.correlation_pnr(images[::5], gSig=3, swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile

In [0]:
#Plot the results of the correlation/PNR projection
plt.figure(figsize=(20,10))
plt.subplot(2, 2, 1); plt.imshow(cn_filter); plt.colorbar(); plt.title('Correlation projection')
plt.subplot(2, 2, 2); plt.imshow(pnr); plt.colorbar(); plt.title('PNR')

In [0]:
# Parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None
gSig = (3, 3)       # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = (15, 15)     # average diameter of a neuron, in general 4*gSig+1
Ain = None          # possibility to seed with predetermined binary masks
merge_thr = .65     # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 20    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 1            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = 1             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
min_corr = .8      # min peak value from correlation image
min_pnr = 8        # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor
memory_fact = 0.8 # How much memory to allocate. 1 works for 16Gb, so 0.8 show be optimized for 12Gb.

opts.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'memory_fact': memory_fact,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True,                # whether to remove duplicates from initialization
                                'border_pix': bord_px})                # number of pixels to not consider in the borders)

# Perform CNMFe extraction
This will take a while. Coffee time!

In [0]:
start = time.time()
# Perform CNMF
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
cnm.fit(images)

end = time.time()
print(end - start)

# Evaluate components

In [0]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

min_SNR = 3            # adaptive way to set threshold on the transient size
r_values_min = 0.85    # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': False})
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

# Plot the results for inspection

In [0]:
%matplotlib inline

#How many neurons to plot
neuronsToPlot = 20

DeconvTraces = cnm.estimates.S
RawTraces = cnm.estimates.C
SFP = cnm.estimates.A
SFP_dims = list(dims)
SFP_dims.append(SFP.shape[1]) 
print('Spatial foootprints dimensions (height x width x neurons): ' + str(SFP_dims))

numNeurons = SFP_dims[2]

SFP = np.reshape(SFP.toarray(), SFP_dims, order='F')

maxRawTraces = np.amax(RawTraces)

plt.figure(figsize=(30,15))
plt.subplot(341);
plt.subplot(345); plt.plot(mc.shifts_rig); plt.title('Motion corrected shifts')
plt.subplot(3,4,9);
plt.subplot(3,4,2); plt.imshow(cn_filter); plt.colorbar(); plt.title('Correlation projection')
plt.subplot(3,4,6); plt.imshow(pnr); plt.colorbar(); plt.title('PNR')
plt.subplot(3,4,10); plt.imshow(np.amax(SFP,axis=2)); plt.colorbar(); plt.title('Spatial footprints')

plt.subplot(2,2,2); plt.figure; plt.title('Example traces (first 50 cells)')
plot_gain = 10 # To change the value gain of traces
if numNeurons >= neuronsToPlot:
  for i in range(neuronsToPlot):
    if i == 0:
      plt.plot(RawTraces[i,:],'k')
    else:
      trace = RawTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')
else:
  for i in range(numNeurons):
    if i == 0:
      plt.plot(RawTraces[i,:],'k')
    else:
      trace = RawTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')

plt.subplot(2,2,4); plt.figure; plt.title('Deconvolved traces (first 50 cells)')
plot_gain = 20 # To change the value gain of traces
if numNeurons >= neuronsToPlot:
  for i in range(neuronsToPlot):
    if i == 0:
      plt.plot(DeconvTraces[i,:],'k')
    else:
      trace = DeconvTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')
else:
  for i in range(numNeurons):
    if i == 0:
      plt.plot(DeconvTraces[i,:],'k')
    else:
      trace = DeconvTraces[i,:] + maxRawTraces*i/plot_gain
      plt.plot(trace,'k')      

# Save summary figure
plt.savefig(dirExperimentName + '/' + 'summary_figure.svg', edgecolor='w', format='svg', transparent=True)

# Register the timestamps for analysis

In [0]:
# Open timestamps.dat here
try:
  metadata, res = dbx.files_download(path_to_analyze + '/' + 'timestamp.dat')
  f = open('timestamp.dat','wb')
  f.write(res.content)
  f.close
except FileExistsError:
  print('Error: timestamp.dat could not be found')

with open('timestamp.dat') as f:
  camNum, frameNum, sysClock, buffer = np.loadtxt(f, dtype='float', comments='#', skiprows=1, unpack = True)
  
cameraMatched = False
for j in range(int(max(camNum))+1):
  if sum(camNum==j) != 0:
    camFrameList, = np.where(camNum == j)
    camLastFrame = camFrameList[-1]
    LastFrame = frameNum[camLastFrame]
    if (sum(camNum==j) == len(RawTraces[1])) and (LastFrame == len(RawTraces[1])):
      camNumber = j
      mstime_idx = np.where(camNum == j)
      mstime=sysClock[mstime_idx]
      mstime[0] = 0
      maxBufferUsed = max(buffer[mstime_idx])
      cameraMatched = True
      
if cameraMatched is not True:
  print('Problem matching up timestamps for ' + experimentName)
else:
  print('Successfully registered timestamps for ' + experimentName)

In [0]:
# Stop counter and register analysis time
analysis_end = time.time()

analysis_duration = analysis_end - analysis_start

print('Done analyzing. This took a total ' + str(analysis_duration) + ' s')

# Save the results in HDF5 format

In [0]:
if save_hdf5: 
  cnm.save('dirExperimentName' + 'analysis_results.hdf5')

# Save the results in Matlab format

In [0]:
if save_mat:
  from scipy.io import savemat
  
  results_dict = {
                'dirName': path_to_analyze,
                'numFiles': len(msFileList),
                'framesNum': len(RawTraces[1]),
                'maxFramesPerFile': 1000,
                'height': dims[0],
                'width': dims[1],
                'Experiment': experimentName,
                'ExperimentTimestamp': experiment_timestamp,
                'camNumber': 0,
                'time': mstime,
                'analysis_time': analysis_time,
                'ds': spatial_downsampling,
                'shifts': mc.shifts_rig,
                'meanFrame': [], #TO DO
                'Centroids': [], #TO DO
                'CorrProj': cn_filter,
                'PeakToNoiseProj': pnr,
                'FiltTraces': [], #TO DO
                'RawTraces': RawTraces.conj().transpose(), #swap time x neurons dimensions
                'SFP': SFP,
                'numNeurons': SFP_dims[2],
                'analysis_duration': analysis_duration
                }

  SFPperm = np.transpose(SFP,[2,0,1])
  sio.savemat(dirExperimentName + '/SFP.mat', {'SFP': SFPperm})
  sio.savemat(dirExperimentName + '/ms.mat', {'ms': results_dict})

# Transfer analysis to Dropbox

In [0]:
with open(dirExperimentName + '/ms.mat', 'rb') as f:
  dbx.files_upload(f.read(), path_to_results + analysis_time + '/' + experimentName + '/ms.mat',mode=dropbox.files.WriteMode("overwrite"))
  
with open(dirExperimentName + '/SFP.mat', 'rb') as f:
  dbx.files_upload(f.read(), path_to_results + analysis_time + '/' + experimentName + '/SFP.mat',mode=dropbox.files.WriteMode("overwrite"))

with open(dirExperimentName + '/summary_figure.png', 'rb') as f:
  dbx.files_upload(f.read(), path_to_results + analysis_time + '/' + experimentName + '/summary_figure.png',mode=dropbox.files.WriteMode("overwrite"))

In [0]:
# Make sure the file is up there (in the cloud)
response = dbx.files_list_folder(path_to_results + analysis_time + '/' + experimentName)

print('These files have successfully been uploaded:')
for file in response.entries:
  print(file.name)

In [0]:
#%% STOP CLUSTER and clean up log files
cm.stop_server(dview=dview)

# **Analyze behavior videos**
This section is to analyse the behavior video present in your experiment folder. Briefly, you will download the videos on your Colab virtual machine, concatenate the video (and optionally crop them), and finally process them with DeepLabCut to perform pose estimation. The output can be saved in a hdf5 format, a Matlab file or simply a csv file. Results are transfered back to your Dropbox/Google drive 

In [0]:
!pip install deeplabcut
import os
os.environ["DLClight"]="True"
os.environ["Colab"]="True"

import deeplabcut

In [0]:
# Run this cell if you have any issue with pillow/register_extensions when plotting images
!pip install Pillow==4.0.0

from PIL import Image
def register_extension(id, extension): Image.EXTENSION[extension.lower()] = id.upper()
Image.register_extension = register_extension
def register_extensions(id, extensions): 
  for extension in extensions: register_extension(id, extension)
Image.register_extensions = register_extensions

!pip install ruamel.yaml==0.15
!pip install pandas==0.21.0

In [0]:
metadata, res = dbx.files_download(path_to_DLC_model)
with open('DeepLabCut.zip','wb') as f:
  f.write(res.content)
  
zip_ref = zipfile.ZipFile('DeepLabCut.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [0]:
path_config_file = '/content/DeepLabCut/Guillaume_trained_model/config.yaml'

In [0]:
sessionFilesResponse = dbx.files_list_folder(path_to_analyze)
filesList = []
behavFileList = []

for file in sessionFilesResponse.entries:
  filesList.append(file.name)

for i in filesList[:]:
  if i.startswith('behav') and i.endswith('.avi'):
    behavFileList.append(i)
    
behavFileList = sorted(behavFileList, key=lambda x: int(re.sub('[behavCam.avi]','', x)))

print('Miniscope files in folder:')
print(behavFileList)

if len(behavFileList) == 0:
  print("No behavior avi files found")

In [0]:
print('Downloading behavior videos from Dropbox.....')
for file in behavFileList:
    f = open(dirExperimentName + '/' + file,"wb+") 
    metadata, res = dbx.files_download(path_to_analyze + '/' + file)
    f.write(res.content)
    f.close()
    print(path_to_analyze + '/' + file + '.....done!')
print('Done downloading behavior videos!')

In [0]:
# Create a list of files downloaded on the Colab virtual machine
behavLocalFileList = [dirExperimentName + '/' + s for s in behavFileList]
print(behavLocalFileList)

In [0]:
# Create behavVideo.avi here
clips=[]
for video in behavLocalFileList:
  if video == behavLocalFileList[0]:
    final_clip = VideoFileClip(video)
  else:
    clip = VideoFileClip(video)
    final_clip = concatenate_videoclips([final_clip, clip]) 

behavVideoFilePath = dirExperimentName + '/' + 'behavVideo.avi'
final_clip.write_videofile(behavVideoFilePath,codec='rawvideo')  

In [0]:
%matplotlib inline
# Visualize a frame for potential cropping
clip = VideoFileClip(behavVideoFilePath)
print('Video dimensions (witdh x height):')
behav_dims = clip.size
print(behav_dims)

clip.save_frame(dirExperimentName + '/' + 'behav_frame.png')

img=mpl.image.imread(dirExperimentName + '/' + 'behav_frame.png')
imgplot = plt.imshow(img); plt.title('Original size')

In [0]:
is_crop = False
if is_crop:
  clip = VideoFileClip(behavVideoFilePath)
  cropped_clip = clip.crop(x1=0, y1=50, x2=638, y2=120)
  cropped_clip.write_videofile(behavVideoFilePath + '_CROPPED.avi',codec='rawvideo')
  behavVideoFilePath = behavVideoFilePath + '_CROPPED.avi'

In [0]:
if is_crop:
  # Visualize the croped video
  clip = VideoFileClip(behavVideoFilePath)
  print(clip.size)
  clip.save_frame(dirExperimentName + '/' + 'cropped_behav_frame.png')

  img=mpl.image.imread(dirExperimentName + '/' + 'cropped_behav_frame.png')
  imgplot = plt.imshow(img); plt.title('Cropped video')

In [0]:
behavVideoFilePath = [behavVideoFilePath]

In [0]:
deeplabcut.analyze_videos(path_config_file,behavVideoFilePath, save_as_csv=True)

In [0]:
# Find the results in the h5file
for file in os.listdir(dirExperimentName):
    if file.endswith('.csv'):
      behavResultsCSVFile = os.path.join(dirExperimentName, file)
      
with open(behavResultsCSVFile) as f:
    reader = csv.reader(f)
    scorer = next(reader);
    bodyparts = next(reader);
    coordinates = next(reader);

header = []
for i in range(len(bodyparts)):
  header.append(bodyparts[i] + '_' + coordinates[i]);
  
with open(behavResultsCSVFile) as f:  
  data = np.loadtxt(f, dtype='float', delimiter=',', comments='#', skiprows=3, unpack = True)

# Delete the first column that does not contain relevant information
header.pop(0)
data = np.delete(data, 0, axis=0)


position = dict(zip(header,data))  
  
# Get number of frames
dims_behav = np.shape(data)
behavNumFrames = dims_behav[1]
print('Behavior video has ' + str(behavNumFrames) + ' frames')

In [0]:
# Open timestamps.dat here
try:
  metadata, res = dbx.files_download(path_to_analyze + '/' + 'timestamp.dat')
  f = open('timestamp.dat','wb')
  f.write(res.content)
  f.close
except FileExistsError:
  print('Error: timestamp.dat could not be found')

with open('timestamp.dat') as f:
  
  camNum, frameNum, sysClock, buffer = np.loadtxt(f, dtype='float', comments='#', skiprows=1, unpack = True)
  
cameraMatched = False
for j in range(int(max(camNum))+1):
  if sum(camNum==j) != 0:
    camFrameList, = np.where(camNum == j)
    camLastFrame = camFrameList[-1]
    LastFrame = frameNum[camLastFrame]
    if (sum(camNum==j) == behavNumFrames) and (LastFrame == behavNumFrames):
      camNumber = j
      behavtime_idx = np.where(camNum == j)
      behavtime=sysClock[behavtime_idx]
      behavtime[0] = 0
      maxBufferUsed = max(buffer[behavtime_idx])
      cameraMatched = True
      
if cameraMatched is not True:
  print('Problem matching up timestamps for ' + experimentName)
else:
  print('Successfully registered timestamps for ' + experimentName)

In [0]:
if save_mat:
  from scipy.io import savemat
  
  results_dict = {
                'dirName': path_to_analyze,
                'numFiles': len(behavFileList),
                'framesNum': behavNumFrames,
                'maxFramesPerFile': 1000,
                'height': behav_dims[1],
                'width': behav_dims[0],
                'Experiment': experimentName,
                'ExperimentTimestamp': experiment_timestamp,
                'camNumber': camNumber,
                'time': behavtime,
                'analysis_time': analysis_time,
                'background': [], #TO DO
                'position': position
                }

  sio.savemat(dirExperimentName + '/behav.mat', {'behav': results_dict})

In [0]:
# This generates a labeled video. You can use it to validate the inference
deeplabcut.create_labeled_video(path_config_file,behavVideoFilePath)

for file in os.listdir(dirExperimentName):
    if file.endswith('.mp4'):
      labeled_video = os.path.join(dirExperimentName, file)
      print(labeled_video)

In [0]:
with open(dirExperimentName + '/behav.mat', 'rb') as f:
  dbx.files_upload(f.read(), path_to_results + analysis_time + '/' + experimentName + '/behav.mat',mode=dropbox.files.WriteMode("overwrite"))
  
with open(labeled_video, 'rb') as f:
  dbx.files_upload(f.read(), path_to_results + analysis_time + '/' + experimentName + '/labeled_video.mp4',mode=dropbox.files.WriteMode("overwrite"))

In [0]:
# Make sure the file is up there (in the cloud)
response = dbx.files_list_folder(path_to_results + analysis_time + '/' + experimentName)

print('These files have successfully been uploaded:')
for file in response.entries:
  print(file.name)

# Done analyzing!
From there, you can program an alarm (eg email, text) to notify you that the analysis has been completed.

In [0]:
msg = 'Your analysis ' + experimentName + ' has just finished!'

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(alert_email, alert_email_password)
 
msg = "COLAB WORK FINISH ALERT!"
server.sendmail(alert_email, alert_email, msg)
server.quit()